# SETI CNN using TF and Binary DS

In [1]:
import requests
import json
#import ibmseti
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import pickle
import time
#!sudo pip install sklearn
from sklearn.metrics import confusion_matrix

### import dataset reader

In [2]:
!wget --output-document SETI.zip  https://ibm.box.com/shared/static/jhqdhcblhua5dx2t7ixwm88okitjrl6l.zip
!unzip -o SETI.zip
import SETI

--2017-05-20 18:29:17--  https://ibm.box.com/shared/static/jhqdhcblhua5dx2t7ixwm88okitjrl6l.zip
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/shared/static/jhqdhcblhua5dx2t7ixwm88okitjrl6l.zip [following]
--2017-05-20 18:29:18--  https://ibm.ent.box.com/shared/static/jhqdhcblhua5dx2t7ixwm88okitjrl6l.zip
Resolving ibm.ent.box.com (ibm.ent.box.com)... 107.152.27.211
Connecting to ibm.ent.box.com (ibm.ent.box.com)|107.152.27.211|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://public.boxcloud.com/d/1/UUAVgkHURnsZeHwZli3Emz0Hn2G5xGzLPOQJ3Cg0hmFnbRlUcQHQZlGvQGqos8slWPqki5hCVzEkivljMk70R0z5vTeu5D1_5TjGOW8lCECzmmUk5F9TNDVP95E90Ecm39EhmHkUKezjUumiBGtY7uVxRi_PFbQJzdvs_jPoaE4GqEEO4NRNCxxTYDrBI5S5NxhwBZTpyCKy-2z0AkEuJY5sWLzJH-QLQu5YFmOSVbXEEIzcLm-vchuWXadvWQIjCT1DyP2oLS9Ck2U1GvPIisReA

### Download data

In [3]:
# The code was removed by DSX for sharing.

mkdir: cannot create directory ‘SETI’: File exists
--2017-05-20 18:29:20--  https://ibm.box.com/shared/static/ww315shk648d3mt3ljbyl1jjte8nkmvj.gz
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/shared/static/ww315shk648d3mt3ljbyl1jjte8nkmvj.gz [following]
--2017-05-20 18:29:20--  https://ibm.ent.box.com/shared/static/ww315shk648d3mt3ljbyl1jjte8nkmvj.gz
Resolving ibm.ent.box.com (ibm.ent.box.com)... 107.152.27.211
Connecting to ibm.ent.box.com (ibm.ent.box.com)|107.152.27.211|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://public.boxcloud.com/d/1/kymwvgV8yFhM88EZMyZP9v6HTTwfqwtJUDCMva0cdzBHkBgcSG_sqDItfhPFotw-Sc8meYFZkPgGbmZKyJhQXRiZTVtCOBZbZBzg-NhWY0pC8qQP0vRZBU48mDP4XYl-AMh9Flrofc5PjfX2GMtPJGFlDWiqEqHOk5S7b_Zd0Doyn8mttvAsVxW22l0owL3uz4ffNWtMG3wbtGSDlWzNLtBMw_6AzFSu04KMXjTiH

### Load data

In [38]:
from tensorflow.examples.tutorials.mnist import input_data
dataset = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
!ls SETI/SETI_ds_256x2048/

test-images-idx3-ubyte.gz  train-images-idx3-ubyte.gz
test-labels-idx1-ubyte.gz  train-labels-idx1-ubyte.gz


In [8]:
dataset = SETI.read_data_sets('SETI/SETI_ds_256x2048/', one_hot=True, validation_size=0)
dataset.train.num_examples

Extracting SETI/SETI_ds_256x2048/train-images-idx3-ubyte.gz
Extracting SETI/SETI_ds_256x2048/train-labels-idx1-ubyte.gz
Extracting SETI/SETI_ds_256x2048/test-images-idx3-ubyte.gz
Extracting SETI/SETI_ds_256x2048/test-labels-idx1-ubyte.gz


694

In [10]:
dataset.train.images.shape

(694, 524288)

## CNN

In [11]:
# Parameters
learning_rate = 0.001
max_training_iters = 500
batch_size = 128
display_step = 10

# Network Parameters

n_classes = 4 # number of possible classifications for the problem
dropout = 0.75 # Dropout, probability to keep units

height = 256 # height of the image in pixels -- 128
width = 2048 # width of the image in pixels -- 1536
n_input = width * height # number of pixels in one image  -196608


In [12]:
x  = tf.placeholder(tf.float32, shape=[None, n_input])
y_ = tf.placeholder(tf.float32, shape=[None, n_classes])

In [13]:
x_image = tf.reshape(x, [-1,height,width,1]) 
x_image

<tf.Tensor 'Reshape:0' shape=(?, 256, 2048, 1) dtype=float32>

#### Convolutional Layer 1

In [14]:
W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1))
b_conv1 = tf.Variable(tf.constant(0.1, shape=[32])) # need 32 biases for 32 outputs
convolve1= tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1
h_conv1 = tf.nn.relu(convolve1)
h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') #max_pool_2x2
layer1= h_pool1
layer1

<tf.Tensor 'MaxPool:0' shape=(?, 128, 1024, 32) dtype=float32>

#### Convolutional Layer 2

In [15]:
W_conv2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))
b_conv2 = tf.Variable(tf.constant(0.1, shape=[64])) #need 64 biases for 64 outputs
convolve2= tf.nn.conv2d(layer1, W_conv2, strides=[1, 1, 1, 1], padding='SAME')+ b_conv2
h_conv2 = tf.nn.relu(convolve2)
h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') #max_pool_2x2
layer2= h_pool2
layer2

<tf.Tensor 'MaxPool_1:0' shape=(?, 64, 512, 64) dtype=float32>

#### Convolutional Layer 3

In [18]:
W_conv3 = tf.Variable(tf.truncated_normal([5, 5, 64, 128], stddev=0.1))
b_conv3 = tf.Variable(tf.constant(0.1, shape=[128])) #need 64 biases for 64 outputs
convolve3= tf.nn.conv2d(layer2, W_conv3, strides=[1, 1, 1, 1], padding='SAME')+ b_conv3
h_conv3 = tf.nn.relu(convolve3)
h_pool3 = tf.nn.max_pool(h_conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') #max_pool_2x2
layer3= h_pool3
layer3

<tf.Tensor 'MaxPool_2:0' shape=(?, 32, 256, 128) dtype=float32>

#### Convolutional Layer 4

In [ ]:
W_conv3 = tf.Variable(tf.truncated_normal([5, 5, 128, 258], stddev=0.1))
b_conv3 = tf.Variable(tf.constant(0.1, shape=[256])) #need 64 biases for 64 outputs
convolve3= tf.nn.conv2d(layer2, W_conv3, strides=[1, 1, 1, 1], padding='SAME')+ b_conv3
h_conv3 = tf.nn.relu(convolve3)
h_pool3 = tf.nn.max_pool(h_conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') #max_pool_2x2
layer3= h_pool3
layer3

#### Fully Connected Layer

In [23]:
input_layer = layer3
dim = input_layer.get_shape().as_list()
dim

[None, 32, 256, 128]

In [ ]:
1048576

In [24]:
dims= dim[1]*dim[2]*dim[3]
prv_layer_matrix = tf.reshape(input_layer, [-1, dims])
W_fc1 = tf.Variable(tf.truncated_normal([dims, 1024], stddev=0.1))
b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024])) # need 1024 biases for 1024 outputs
fcl1  = tf.matmul(prv_layer_matrix, W_fc1) + b_fc1
h_fc1 = tf.nn.relu(fcl1) # ???
h_fc1


<tf.Tensor 'Relu_4:0' shape=(?, 1024) dtype=float32>

In [25]:
keep_prob = tf.placeholder(tf.float32)
layer_drop = tf.nn.dropout(h_fc1, keep_prob)

#### Classification Layer

In [26]:
W_fc2 = tf.Variable(tf.truncated_normal([1024, n_classes], stddev=0.1)) #1024 neurons
b_fc2 = tf.Variable(tf.constant(0.1, shape=[n_classes])) # 10 possibilities for classes [0,1,2,3]
fcl2 = tf.matmul(layer_drop, W_fc2) + b_fc2
y_CNN= tf.nn.softmax(fcl2)

#### Training


In [27]:
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_l4_conv), reduction_indices=[1]))
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_CNN, labels=y_))
train_step = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

In [28]:
correct_prediction = tf.equal(tf.argmax(y_CNN,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [29]:
# Initializing the variables
init = tf.initialize_all_variables()


In [30]:
# Launch the graph
with tf.Session() as sess:
    X_test = dataset.test.images
    y_test = dataset.test.labels
    sess.run(init)
    step = 1
    while step < max_training_iters:
    # Keep training until reach max iterations
        
        start = time.time()

        x_batch, y_batch = dataset.train.next_batch(batch_size)
        # Run optimization op (backprop)
        #sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
        train_step.run(feed_dict={x: x_batch, y_: y_batch, keep_prob: dropout})

        end = time.time()

        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cross_entropy, accuracy], feed_dict={x: x_batch,y_: y_batch,keep_prob: 1.})
            #train_accuracy = accuracy.eval(feed_dict={x:x_batch, y_: y_batch,  keep_prob: 0.5})
            
            
            test_accuracy = sess.run(accuracy, feed_dict={x: X_test, y_: y_test, keep_prob: 1.})
            
            print("Iter " + str(step) + \
                ", Training time= " + "{:.5f}".format(end - start) + \
                ", Minibatch Loss= " +  "{:.6f}".format(loss) +  \
                ", Training Accuracy= " + "{:.5f}".format(acc)  + \
                ", Test Accuracy= " + "{:.5f}".format(test_accuracy) )
        step += 1
    print("Optimization Finished!")
    
    
    
    # Calculate accuracy for test images
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: X_test, y_: y_test, keep_prob: 1.}))
        
    # Find the labels of test set
    y = sess.run(tf.argmax(y_l4_conv,1), feed_dict={x: X_test, y_: y_test, keep_prob: 1.})
    
    # lets save kernels
    #kernels_l1 = sess.run(tf.reshape(tf.transpose(W_conv1, perm=[2, 3, 0, 1]),[32,-1]))
    #kernels_l2 = sess.run(tf.reshape(tf.transpose(W_conv2, perm=[2, 3, 0, 1]),[32*64,-1]))

ResourceExhaustedError: OOM when allocating tensor with shape[1048576,1024]
	 [[Node: Variable_8/Adam_1/Assign = Assign[T=DT_FLOAT, _class=["loc:@Variable_8"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/cpu:0"](Variable_8/Adam_1, zeros_13)]]
Caused by op u'Variable_8/Adam_1/Assign', defined at:
  File "/usr/local/src/bluemix_jupyter_bundle.v44/notebook/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/local/src/bluemix_jupyter_bundle.v44/notebook/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/src/bluemix_jupyter_bundle.v44/notebook/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/src/bluemix_jupyter_bundle.v44/notebook/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/usr/local/src/bluemix_jupyter_bundle.v

## Evaluation

In [ ]:
y_ = np.argmax(y_test,1)
confusion_matrix(y_, y)

### Viz

In [ ]:
!wget --output-document utils1.py http://deeplearning.net/tutorial/code/utils.py
import utils1
from utils1 import tile_raster_images

In [ ]:
#from utils import tile_raster_images
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline
image = Image.fromarray(tile_raster_images(kernels_l1, img_shape=(5, 5) ,tile_shape=(4, 8), tile_spacing=(1, 1)))
### Plot image
plt.rcParams['figure.figsize'] = (18.0, 18.0)
imgplot = plt.imshow(image)
imgplot.set_cmap('gray')  

In [17]:
image = Image.fromarray(tile_raster_images(kernels_l2, img_shape=(5, 5) ,tile_shape=(4, 12), tile_spacing=(1, 1)))
### Plot image
plt.rcParams['figure.figsize'] = (18.0, 18.0)
imgplot = plt.imshow(image)
imgplot.set_cmap('gray')  

NameError: name 'Image' is not defined